# **Salomon Uran Parra C.C. 1015068767**

## **Laboratorio 8 - Aprendizaje Estadistico**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons, make_regression, make_circles, make_blobs, load_digits

### **1. Para el dataset make a moon de sklearn, construir un modelo de una red neuronal con keras para clasificar los datos.**

Vamos a crear y visualizar primero el dataset:

In [ ]:
X_moons, y_moons = make_moons(n_samples=1000, noise=0.1, random_state=42) #make_moons permite crear un dataset y clasificarlo para dos figuras en forma de luna

print("Dimensiones de los datos (X_moons):", X_moons.shape)
print("Dimensiones de las etiquetas (y_moons):", y_moons.shape)
print("Tipos de datos de las características (X_moons):", X_moons.dtype)
print("Valores únicos en las etiquetas (y_moons):", np.unique(y_moons))


plt.figure(figsize=(8, 6))
plt.scatter(X_moons[y_moons == 0, 0], X_moons[y_moons == 0, 1], label='Clase 0', alpha=0.7)
plt.scatter(X_moons[y_moons == 1, 0], X_moons[y_moons == 1, 1], label='Clase 1', alpha=0.7)
plt.title('Dataset Make Moons')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.legend()
plt.grid(True)
plt.show()

Ahora, vamos a crear una red neuronal densa con output Sigmoide que permita clasificar entre la clase 0 (azul) y la clase 1 (naranja) empleando el API Keras de Tensorflow. Definimos el modelo con dos capas ocultas de 4 y 8 neuronas ReLu, similar al modelo de clasificacion de los circulos hecho en el collab de la sesion de Keras. El output sera una funcion sigmoide y se usara una funcion de perdida Binary Cross Entropy:

In [ ]:
def build_model_moons(activation = 'sigmoid', loss='binary_crossentropy'):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(4, input_dim=2, activation='relu'))
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation = activation))
    opt = keras.optimizers.SGD(learning_rate=0.01)
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model

Ahora podemos proceder a entrenar un modelo de clasificacion empleando esta red neuronal. Vamos a disponer del 50% de los datos para entrenamiento y el otro 50% para test, de forma analoga a la clasificacion de los circulos vista en clase. Se entrenaran unicamente 100 epocas, pues el modelo con dos capas ocultas resulta lo suficientemente robusto para lograr una buena precision en pocas epocas de entrenamiento.

In [ ]:
keras.backend.clear_session()

n_train = 500 #numero de datos dedicados a entrenamiento de los 1000 datos obtenidos del make_moons

train_X, test_X = X_moons[:n_train, :], X_moons[n_train:, :]
train_y, test_y = y_moons[:n_train], y_moons[n_train:]
#seleccion de conjuntos de entrenamiento y testeo

model = build_model_moons()
history = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=100, verbose=1)
#se construye el modelo y el historial de entrenamiento que incluye
#la evolucion de la funcion de perdida tanto en el entrenamiento como testeo, y asi mismo la precision en ambos conjuntos

_, train_acc = model.evaluate(train_X, train_y, verbose=0)
_, test_acc = model.evaluate(test_X, test_y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
#se imprime tanto la precision en el conjunto de entrenamiento como en el de testeo para cada epoca

plt.figure(figsize=(14,7))

plt.subplot(121)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')

plt.subplot(122)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

Ahora por ejemplo, podemos ver como clasifca puntos en el plano. De la imagen original de los datos podemos ver que el punto $(0,1)$ pertenece a la clase 0 mientras que el punto $(1,-0.5)$ pertenece a la clase 1. Veamos que resultado arroja el modelo a la hora de predecir la clase a la que pertenecen:

In [ ]:
#clase 0 se clasifica como un numero real menor que 0.5 y mayor que cero, mientras que la clase 1 se clasifica como un real entre 0.5 y 1
print(model.predict(np.array([[0,1]])))

print(model.predict(np.array([[1,-0.5]])))

Con esto vemos que el modelo tiene un buen desempeño a la hora de clasificar puntos que tengan una pertenencia clara a los conjuntos definidos por el dataset. Faltaria analizar la precision que tiene en puntos cercanos a la "frontera" que hay entre las dos clases.

### **2. Para el dataset load digits, construir un modelo de red neuronal empleando keras para realizar la clasificación.**

Primero visualizaremos el dataset de load_digits, utilizando unicamente los primeros 6 digitos, es decir los enteros del 0 al 5.

In [ ]:
digits = load_digits(n_class=6)
#se lee el dataset con solo los digitos del 0 a 5

X, y = digits.data, digits.target
n_samples, n_features = X.shape
#se leen los datos y su target o label (la clasificacion de los datos por digitos)

fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(6, 6))

for idx, ax in enumerate(axs.ravel()):
    ax.imshow(X[idx].reshape((8, 8)), cmap=plt.cm.binary)
    ax.axis("off")
_ = fig.suptitle("Algunos de los datos de imagenes en el dataset load_digits", fontsize=16)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#hacemos un splitting aleatorio de los datos de entrenamiento y test usando train_test_split

Ahora haremos un modelo de clasificacion similar al del punto anterior pero con las siguientes diferencias: Se emplearan dos capas ocultas de 8 y 16 neuronas ReLu, se usara como output una funcion SoftMax para el multiclasificador y se empleara una funcion de coste que generaliza la binary cross entropy, llamada sparse categorical crossentropy. Nuevamente se hace uso del API Keras.

In [ ]:
def build_model_digits(activation = 'softmax', loss='sparse_categorical_crossentropy'): #clasificacion binaria
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(8, input_dim = 64 , activation='relu'))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(6, activation = activation))
    model.compile( loss=loss, optimizer='adam', metrics=['accuracy'])
    return model

Ahora, como del dataset ya se obtuvo previamente unos conjuntos de entrenamiento y test, simplemente definiremos el modelo utilizando 100 epocas de entrenamiento e imprimiremos nuevamente toda la informacion acerca del entrenamiento como antes.

In [ ]:
keras.backend.clear_session()
model = build_model_digits()
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=1)
#se construye el modelo y el historial de entrenamiento nuevamente

_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_train, y_train, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
#se imprime tanto la precision en el conjunto de entrenamiento como en el de testeo para cada epoca

plt.figure(figsize=(14,7))

plt.subplot(121)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')

plt.subplot(122)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')

plt.legend()
plt.show()

El modelo se entreno de forma muy eficiente y precisa, logrando una precision de mas del 90% en ambos conjuntos. Podemos visualizar como predice la red neuronal a continuacion, con un ejemplo del digito 5.

In [ ]:
plt.imshow(X[11].reshape((8, 8)), cmap=plt.cm.binary)

In [ ]:
prds = model.predict(X)
prds[11]

Vemos que el output mas grande esta justamente en el indice 5, es decir, el modelo predice que la imagen "luce" como un cinco con la mayor probabilidad.